# Del 03: Web Communication and Web Scraping

## HTTP REST APIs (requests library)

https://realpython.com/python-requests/

The requests library is the de facto standard for making HTTP requests in Python. It abstracts the complexities of making requests behind a beautiful, simple API so that you can focus on interacting with services and consuming data in your application.

### Getting Started With requests

- Ustvarimo novo virtualno okolje:

    python3.8 -m venv .venv

- V VS Code izberemo virtulano okolje

- Let’s begin by installing the requests library. To do so, run the following command:
    - Ustvarimo `requirements.txt` in dodamo zadnjo verzijo knjižnice
    - `requests==2.24.0`
    -  `pip install -r requirements.txt`

Once requests is installed, you can use it in your application. Importing requests looks like this:

In [2]:
import requests

Now that you’re all set up, it’s time to begin your journey through requests. Your first goal will be learning how to make a GET request.

### The GET Request

HTTP methods such as GET and POST, determine which action you’re trying to perform when making an HTTP request. Besides GET and POST, there are several other common methods that you’ll use later in this tutorial.

One of the most common HTTP methods is GET. The GET method indicates that you’re trying to get or retrieve data from a specified resource. To make a GET request, invoke requests.get().

To test this out, you can make a GET request to GitHub’s Root REST API by calling get() with the following URL:





In [14]:
url = 'https://jsonplaceholder.typicode.com/users'

In [5]:
requests.get(url)

<Response [200]>

Congratulations! You’ve made your first request. Let’s dive a little deeper into the response of that request.

### The Response

A Response is a powerful object for inspecting the results of the request. Let’s make that same request again, but this time store the return value in a variable so that you can get a closer look at its attributes and behaviors:

In [6]:
response = requests.get(url)

In this example, you’ve captured the return value of get(), which is an instance of Response, and stored it in a variable called response. You can now use response to see a lot of information about the results of your GET request.

#### Status Codes

The first bit of information that you can gather from Response is the status code. A status code informs you of the status of the request.

For example, a 200 OK status means that your request was successful, whereas a 404 NOT FOUND status means that the resource you were looking for was not found. There are many other possible status codes as well to give you specific insights into what happened with your request.

By accessing .status_code, you can see the status code that the server returned:

In [7]:
response.status_code

200

.status_code returned a 200, which means your request was successful and the server responded with the data you were requesting.

Sometimes, you might want to use this information to make decisions in your code:

In [9]:
if response.status_code == 200:
    print('Success!')
elif response.status_code == 404:
    print('Not Found.')

Success!


With this logic, if the server returns a 200 status code, your program will print Success!. If the result is a 404, your program will print Not Found.

requests goes one step further in simplifying this process for you. If you use a Response instance in a conditional expression, it will evaluate to True if the status code was between 200 and 400, and False otherwise.

Therefore, you can simplify the last example by rewriting the if statement:



In [10]:
if response:
    print('Success!')
else:
    print('An error has occurred.')

Success!


> Technical Detail: This Truth Value Test is made possible because __bool__() is an overloaded method on Response. This means that the default behavior of Response has been redefined to take the status code into account when determining the truth value of the object.

Keep in mind that this method is not verifying that the status code is equal to 200. The reason for this is that other status codes within the 200 to 400 range, such as 204 NO CONTENT and 304 NOT MODIFIED, are also considered successful in the sense that they provide some workable response.

For example, the 204 tells you that the response was successful, but there’s no content to return in the message body.

So, make sure you use this convenient shorthand only if you want to know if the request was generally successful and then, if necessary, handle the response appropriately based on the status code.

Let’s say you don’t want to check the response’s status code in an if statement. Instead, you want to raise an exception if the request was unsuccessful. You can do this using .raise_for_status():

In [11]:
import requests
from requests.exceptions import HTTPError

for url in [url, url + '/invalid']:
    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        print('Success!')

Success!
HTTP error occurred: 404 Client Error: Not Found for url: https://jsonplaceholder.typicode.com/users/invalid


If you invoke .raise_for_status(), an HTTPError will be raised for certain status codes. If the status code indicates a successful request, the program will proceed without that exception being raised.

Now, you know a lot about how to deal with the status code of the response you got back from the server. However, when you make a GET request, you rarely only care about the status code of the response. Usually, you want to see more. Next, you’ll see how to view the actual data that the server sent back in the body of the response.

#### Content

The response of a GET request often has some valuable information, known as a payload, in the message body. Using the attributes and methods of Response, you can view the payload in a variety of different formats.

To see the response’s content in bytes, you use .content:

In [15]:
response = requests.get(url)
response.content[:250] #odstrani zares

b'[\n  {\n    "id": 1,\n    "name": "Leanne Graham",\n    "username": "Bret",\n    "email": "Sincere@april.biz",\n    "address": {\n      "street": "Kulas Light",\n      "suite": "Apt. 556",\n      "city": "Gwenborough",\n      "zipcode": "92998-3874",\n      "ge'

While .content gives you access to the raw bytes of the response payload, you will often want to convert them into a string using a character encoding such as UTF-8. response will do that for you when you access .text:

In [18]:
print(response.text[:250])

[
  {
    "id": 1,
    "name": "Leanne Graham",
    "username": "Bret",
    "email": "Sincere@april.biz",
    "address": {
      "street": "Kulas Light",
      "suite": "Apt. 556",
      "city": "Gwenborough",
      "zipcode": "92998-3874",
      "ge


Because the decoding of bytes to a str requires an encoding scheme, requests will try to guess the encoding based on the response’s headers if you do not specify one. You can provide an explicit encoding by setting .encoding before accessing .text:

In [19]:
response.encoding = 'utf-8' # Optional: requests infers this internally
print(response.text[:250])

[
  {
    "id": 1,
    "name": "Leanne Graham",
    "username": "Bret",
    "email": "Sincere@april.biz",
    "address": {
      "street": "Kulas Light",
      "suite": "Apt. 556",
      "city": "Gwenborough",
      "zipcode": "92998-3874",
      "ge


If you take a look at the response, you’ll see that it is actually serialized JSON content. To get a dictionary, you could take the str you retrieved from .text and deserialize it using json.loads(). However, a simpler way to accomplish this task is to use .json():

In [21]:
#response.json()

The type of the return value of .json() is a dictionary, so you can access values in the object by key.

You can do a lot with status codes and message bodies. But, if you need more information, like metadata about the response itself, you’ll need to look at the response’s headers.

#### Headers

The response headers can give you useful information, such as the content type of the response payload and a time limit on how long to cache the response. To view these headers, access .headers:

In [22]:
response.headers

{'Date': 'Wed, 21 Oct 2020 12:55:13 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=d53beab79b0d6f04a45bf8fb1c3622eee1603284913; expires=Fri, 20-Nov-20 12:55:13 GMT; path=/; domain=.typicode.com; HttpOnly; SameSite=Lax', 'X-Powered-By': 'Express', 'X-Ratelimit-Limit': '1000', 'X-Ratelimit-Remaining': '998', 'X-Ratelimit-Reset': '1598842094', 'Vary': 'Origin, Accept-Encoding', 'Access-Control-Allow-Credentials': 'true', 'Cache-Control': 'max-age=43200', 'Pragma': 'no-cache', 'Expires': '-1', 'X-Content-Type-Options': 'nosniff', 'Etag': 'W/"160d-1eMSsxeJRfnVLRBmYJSbCiJZ1qQ"', 'Content-Encoding': 'gzip', 'Via': '1.1 vegur', 'CF-Cache-Status': 'HIT', 'Age': '7972', 'cf-request-id': '05ecd366770000e8eb162c9000000001', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report?lk

.headers returns a dictionary-like object, allowing you to access header values by key. For example, to see the content type of the response payload, you can access Content-Type:

In [23]:
response.headers['Content-Type']

'application/json; charset=utf-8'

There is something special about this dictionary-like headers object, though. The HTTP spec defines headers to be case-insensitive, which means we are able to access these headers without worrying about their capitalization:

In [24]:
response.headers['content-type']

'application/json; charset=utf-8'

Whether you use the key 'content-type' or 'Content-Type', you’ll get the same value.

Now, you’ve learned the basics about Response. You’ve seen its most useful attributes and methods in action. Let’s take a step back and see how your responses change when you customize your GET requests.

### Query String Parameters

One common way to customize a GET request is to pass values through query string parameters in the URL. To do this using get(), you pass data to params. For example, you can use GitHub’s Search API to look for the requests library:

In [29]:
import requests

# Search GitHub's repositories for requests
response = requests.get(
    url,
    params={'id': 8},
)

# Inspect some attributes of the `requests` repository
json_response = response.json()

print(json_response)
repository = json_response[0]
print(f'Person name: {repository["name"]}')  # Python 3.6+
print(f'Company name: {repository["company"]["name"]}')  # Python 3.6+

[{'id': 8, 'name': 'Nicholas Runolfsdottir V', 'username': 'Maxime_Nienow', 'email': 'Sherwood@rosamond.me', 'address': {'street': 'Ellsworth Summit', 'suite': 'Suite 729', 'city': 'Aliyaview', 'zipcode': '45169', 'geo': {'lat': '-14.3990', 'lng': '-120.7677'}}, 'phone': '586.493.6943 x140', 'website': 'jacynthe.com', 'company': {'name': 'Abernathy Group', 'catchPhrase': 'Implemented secondary concept', 'bs': 'e-enable extensible e-tailers'}}]
Person name: Nicholas Runolfsdottir V
Company name: Abernathy Group


By passing the dictionary {'q': 'requests+language:python'} to the params parameter of .get(), you are able to modify the results that come back from the Search API.

You can pass params to get() in the form of a dictionary, as you have just done, or as a list of tuples:

In [31]:
requests.get(
    url,
    params=[('id', 2)],
)

<Response [200]>

You can even pass the values as bytes:

In [32]:
requests.get(
    url,
    params=b'id=2',
)

<Response [200]>

Query strings are useful for parameterizing GET requests. You can also customize your requests by adding or modifying the headers you send.

### Request Headers

To customize headers, you pass a dictionary of HTTP headers to get() using the headers parameter. For example, you can change your previous search request to highlight matching search terms in the results by specifying the text-match media type in the Accept header:

In [36]:
import requests

response = requests.get(
    url,
    params={'id': 3},
    headers={'Accept': 'application/json'},
)

# View the new `text-matches` array which provides information
# about your search term within the results
json_response = response.json()
users = json_response[0]
print(f'Website: {users["website"]}')

Website: ramiro.info


The Accept header tells the server what content types your application can handle. In this case, since you’re expecting the matching search terms to be highlighted, you’re using the header value application/vnd.github.v3.text-match+json, which is a proprietary GitHub Accept header where the content is a special JSON format.

Before you learn more ways to customize requests, let’s broaden the horizon by exploring other HTTP methods.

### Other HTTP Methods

Aside from GET, other popular HTTP methods include POST, PUT, DELETE, HEAD, PATCH, and OPTIONS. requests provides a method, with a similar signature to get(), for each of these HTTP methods:

    requests.post('https://httpbin.org/post', data={'key':'value'})
    requests.post('https://httpbin.org/post', data={'key':'value'})
    requests.put('https://httpbin.org/put', data={'key':'value'})
    requests.delete('https://httpbin.org/delete')
    requests.head('https://httpbin.org/get')
    requests.patch('https://httpbin.org/patch', data={'key':'value'})
    requests.options('https://httpbin.org/get')

Each function call makes a request to the httpbin service using the corresponding HTTP method. For each method, you can inspect their responses in the same way you did before:

In [37]:
response = requests.head('https://httpbin.org/get')
response.headers['Content-Type']

'application/json'

In [41]:
response = requests.delete('https://httpbin.org/delete', params={'id': 8})
json_response = response.json()
json_response

{'args': {'id': '8'},
 'data': '',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '0',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.24.0',
  'X-Amzn-Trace-Id': 'Root=1-5f903642-4e8c9537374d8dd063c7b3f9'},
 'json': None,
 'origin': '212.235.185.131',
 'url': 'https://httpbin.org/delete?id=8'}

Headers, response bodies, status codes, and more are returned in the Response for each method. Next you’ll take a closer look at the POST, PUT, and PATCH methods and learn how they differ from the other request types.

## The Message Body

According to the HTTP specification, POST, PUT, and the less common PATCH requests pass their data through the message body rather than through parameters in the query string. Using requests, you’ll pass the payload to the corresponding function’s data parameter.

data takes a dictionary, a list of tuples, bytes, or a file-like object. You’ll want to adapt the data you send in the body of your request to the specific needs of the service you’re interacting with.

For example, if your request’s content type is application/x-www-form-urlencoded, you can send the form data as a dictionary:

In [42]:
requests.post('https://httpbin.org/post', data={'key':'value'})

<Response [200]>

You can also send that same data as a list of tuples:

In [43]:
requests.post('https://httpbin.org/post', data=[('key', 'value')])

<Response [200]>

If, however, you need to send JSON data, you can use the json parameter. When you pass JSON data via json, requests will serialize your data and add the correct Content-Type header for you.

httpbin.org is a great resource created by the author of requests, Kenneth Reitz. It’s a service that accepts test requests and responds with data about the requests. For instance, you can use it to inspect a basic POST request:

In [44]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})

In [45]:
json_response = response.json()
json_response['data']

'{"key": "value"}'

In [46]:
json_response['headers']['Content-Type']

'application/json'

You can see from the response that the server received your request data and headers as you sent them. requests also provides this information to you in the form of a PreparedRequest.

### Inspecting Your Request

When you make a request, the requests library prepares the request before actually sending it to the destination server. Request preparation includes things like validating headers and serializing JSON content.

You can view the PreparedRequest by accessing .request:

In [47]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})

In [48]:
response.request.headers['Content-Type']

'application/json'

In [49]:
response.request.url

'https://httpbin.org/post'

In [50]:
response.request.body

b'{"key": "value"}'

Inspecting the PreparedRequest gives you access to all kinds of information about the request being made such as payload, URL, headers, authentication, and more.

So far, you’ve made a lot of different kinds of requests, but they’ve all had one thing in common: they’re unauthenticated requests to public APIs. Many services you may come across will want you to authenticate in some way.

### Authentication

Authentication helps a service understand who you are. Typically, you provide your credentials to a server by passing data through the Authorization header or a custom header defined by the service. All the request functions you’ve seen to this point provide a parameter called auth, which allows you to pass your credentials.

One example of an API that requires authentication is GitHub’s Authenticated User API. This endpoint provides information about the authenticated user’s profile. To make a request to the Authenticated User API, you can pass your GitHub username and password in a tuple to get():

In [20]:
from getpass import getpass
requests.get('https://api.github.com/user', auth=('username', getpass()))

········


<Response [403]>

The request succeeded if the credentials you passed in the tuple to auth are valid. If you try to make this request with no credentials, you’ll see that the status code is 401 Unauthorized:



In [21]:
requests.get('https://api.github.com/user')

<Response [401]>

When you pass your username and password in a tuple to the auth parameter, requests is applying the credentials using HTTP’s Basic access authentication scheme under the hood.

Therefore, you could make the same request by passing explicit Basic authentication credentials using HTTPBasicAuth:

In [22]:
from requests.auth import HTTPBasicAuth
from getpass import getpass
requests.get(
    'https://api.github.com/user',
    auth=HTTPBasicAuth('username', getpass())
)

········


<Response [403]>

Though you don’t need to be explicit for Basic authentication, you may want to authenticate using another method. requests provides other methods of authentication out of the box such as HTTPDigestAuth and HTTPProxyAuth.

You can even supply your own authentication mechanism. To do so, you must first create a subclass of AuthBase. Then, you implement `__call__()`:

In [ ]:
import requests
from requests.auth import AuthBase

class TokenAuth(AuthBase):
    """Implements a custom authentication scheme."""

    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        """Attach an API token to a custom auth header."""
        r.headers['X-TokenAuth'] = f'{self.token}'  # Python 3.6+
        return r


requests.get('https://httpbin.org/get', auth=TokenAuth('12345abcde-token'))

Here, your custom TokenAuth mechanism receives a token, then includes that token in the X-TokenAuth header of your request.

Bad authentication mechanisms can lead to security vulnerabilities, so unless a service requires a custom authentication mechanism for some reason, you’ll always want to use a tried-and-true auth scheme like Basic or OAuth.

While you’re thinking about security, let’s consider dealing with SSL Certificates using requests.

### SSL Certificate Verification

Any time the data you are trying to send or receive is sensitive, security is important. The way that you communicate with secure sites over HTTP is by establishing an encrypted connection using SSL, which means that verifying the target server’s SSL Certificate is critical.

The good news is that requests does this for you by default. However, there are some cases where you might want to change this behavior.

If you want to disable SSL Certificate verification, you pass False to the verify parameter of the request function:

In [23]:
requests.get('https://api.github.com', verify=False)

/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<Response [200]>

requests even warns you when you’re making an insecure request to help you keep your data safe!

> Note: requests uses a package called certifi to provide Certificate Authorities. This lets requests know which authorities it can trust. Therefore, you should update certifi frequently to keep your connections as secure as possible.

### Performance

When using requests, especially in a production application environment, it’s important to consider performance implications. Features like timeout control, sessions, and retry limits can help you keep your application running smoothly.

#### Timeouts

When you make an inline request to an external service, your system will need to wait upon the response before moving on. If your application waits too long for that response, requests to your service could back up, your user experience could suffer, or your background jobs could hang.

By default, requests will wait indefinitely on the response, so you should almost always specify a timeout duration to prevent these things from happening. To set the request’s timeout, use the timeout parameter. timeout can be an integer or float representing the number of seconds to wait on a response before timing out:



In [24]:
requests.get('https://api.github.com', timeout=1)

<Response [200]>

In [25]:
requests.get('https://api.github.com', timeout=3.05)

<Response [200]>

In the first request, the request will timeout after 1 second. In the second request, the request will timeout after 3.05 seconds.

You can also pass a tuple to timeout with the first element being a connect timeout (the time it allows for the client to establish a connection to the server), and the second being a read timeout (the time it will wait on a response once your client has established a connection):

In [26]:
requests.get('https://api.github.com', timeout=(2, 5))

<Response [200]>

If the request establishes a connection within 2 seconds and receives data within 5 seconds of the connection being established, then the response will be returned as it was before. If the request times out, then the function will raise a Timeout exception:

In [30]:
import requests
from requests.exceptions import Timeout

try:
    response = requests.get('https://api.github.com', timeout=1)
except Timeout:
    print('The request timed out')
else:
    print('The request did not time out')

The request did not time out


Your program can catch the Timeout exception and respond accordingly.

#### The Session Object

Until now, you’ve been dealing with high level requests APIs such as get() and post(). These functions are abstractions of what’s going on when you make your requests. They hide implementation details such as how connections are managed so that you don’t have to worry about them.

Underneath those abstractions is a class called Session. If you need to fine-tune your control over how requests are being made or improve the performance of your requests, you may need to use a Session instance directly.

Sessions are used to persist parameters across requests. For example, if you want to use the same authentication across multiple requests, you could use a session:



In [32]:
import requests
from getpass import getpass

# By using a context manager, you can ensure the resources used by
# the session will be released after use
with requests.Session() as session:
    session.auth = ('username', getpass())

    # Instead of requests.get(), you'll use session.get()
    response = session.get('https://api.github.com/user')

# You can inspect the response just like you did before
print(response.headers)
print(response.json())

········
{'Date': 'Mon, 17 Feb 2020 21:27:11 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Server': 'GitHub.com', 'Status': '403 Forbidden', 'X-GitHub-Media-Type': 'github.v3; format=json', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '50', 'X-RateLimit-Reset': '1581977544', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'Content-Security-Policy': "default-src 'none'", 'Vary': 'Accept-Encoding, Accept, X-Requested-With', 'Content-Encoding': 'gzip', 'X-GitHub-Request-Id': '0

Each time you make a request with session, once it has been initialized with authentication credentials, the credentials will be persisted.

The primary performance optimization of sessions comes in the form of persistent connections. When your app makes a connection to a server using a Session, it keeps that connection around in a connection pool. When your app wants to connect to the same server again, it will reuse a connection from the pool rather than establishing a new one.

#### Max Retries

When a request fails, you may want your application to retry the same request. However, requests will not do this for you by default. To apply this functionality, you need to implement a custom Transport Adapter.

Transport Adapters let you define a set of configurations per service you’re interacting with. For example, let’s say you want all requests to https://api.github.com to retry three times before finally raising a ConnectionError. You would build a Transport Adapter, set its max_retries parameter, and mount it to an existing Session:

In [34]:
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import ConnectionError

github_adapter = HTTPAdapter(max_retries=3)

session = requests.Session()

# Use `github_adapter` for all requests to endpoints that start with this URL
session.mount('https://api.github.com', github_adapter)

try:
    session.get('https://api.github.com')
except ConnectionError as ce:
    print(ce)

When you mount the HTTPAdapter, github_adapter, to session, session will adhere to its configuration for each request to https://api.github.com.

Timeouts, Transport Adapters, and sessions are for keeping your code efficient and your application resilient.

### Vaja 1: Bicikelj

Podatki: https://opendata.si/promet/bicikelj/list/

In [51]:
url = 'https://opendata.si/promet/bicikelj/list/'

In [52]:
import requests
from requests.exceptions import Timeout

In [53]:
def get_json_data(url):
    try:
        response = requests.get(url, timeout=2)
    except ConnectionError as e:
        print(f'Faild to connect to {url}. Info about a error: {e}')
    except Timeout:
        print('The request timed out')
    else:
        if response.status_code == 200:
            return response.json()
        else:
            print('Not Found.')

In [54]:
bicikelj_data = get_json_data(url)

Izpišemo v postaje:

In [55]:
postaje = [postaja['fullAddress'] for (num, postaja) in bicikelj_data['markers'].items()]

In [56]:
print(postaje)

['PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE', 'POGAČARJEV TRG-TRŽNICA', 'KONGRESNI TRG-ŠUBIČEVA ULICA', 'CANKARJEVA UL.-NAMA', 'BREG', 'GRUDNOVO NABREŽJE-KARLOVŠKA C.', 'BAVARSKI DVOR', 'TRG OF-KOLODVORSKA UL.', 'MASARYKOVA DDC', 'VILHARJEVA CESTA', 'PARK NAVJE-ŽELEZNA CESTA', 'TRG MDB', 'PARKIRIŠČE NUK 2-FF', 'AMBROŽEV TRG', 'GH ŠENTPETER-NJEGOŠEVA C.', 'ILIRSKA ULICA', 'TIVOLI', 'STARA CERKEV', 'KINO ŠIŠKA', 'ŠPICA', 'BARJANSKA C.-CENTER STAREJŠIH TRNOVO', 'ZALOŠKA C.-GRABLOVIČEVA C.', 'TRŽNICA MOSTE', 'ROŽNA DOLINA-ŠKRABČEVA UL.', 'DUNAJSKA C.-PS PETROL', 'PLEČNIKOV STADION', 'DUNAJSKA C.-PS MERCATOR', 'LIDL - VOJKOVA CESTA', 'ŠPORTNI CENTER STOŽICE', 'KOPRSKA ULICA', 'MERCATOR CENTER ŠIŠKA', 'CITYPARK', 'BTC CITY/DVORANA A', 'BTC CITY ATLANTIS', 'TRNOVO', 'P+R BARJE', 'P + R DOLGI MOST', 'BONIFACIJA', 'BRATOVŠEVA PLOŠČAD', 'BS4-STOŽICE', 'SAVSKO NASELJE 2-LINHARTOVA CESTA', 'SAVSKO NASELJE 1-ŠMARTINSKA CESTA', 'SITULA', 'ŠTEPANJSKO NASELJE 1-JAKČEVA ULICA', 'HOFER-KAJUHOVA', 'BRODARJEV TR

Število stevilo_koles in prostih mest:

In [57]:
stevilo_koles = sum([int(postaja['station']['available']) for (num, postaja) in bicikelj_data['markers'].items()])

In [58]:
stevilo_koles

474

In [59]:
prosta_mesta = sum([int(postaja['station']['free']) for (num, postaja) in bicikelj_data['markers'].items()])

In [60]:
prosta_mesta

687

## Vaja 2: BGPView API

BGPView is a simple API allowing consumers to view all sort of analytics data about the current state and structure of the internet.

Stan: https://bgpview.docs.apiary.io/

In [61]:
asn_list = [135340, 43350, 24940, 34305]
url = 'https://api.bgpview.io/search'

In [62]:
def get_asn_data(url, asn_number):
    try:
        params={'query_term': str(asn_number)}
        response = requests.get(url, params=params, timeout=2)
    except ConnectionError as e:
        print(f'Faild to connect to {url}. Info about a error: {e}')
    except Timeout:
        print('The request timed out')
    else:
        if response.status_code == 200:
            return response.json()
        else:
            print('Not Found.')

In [63]:
for asn_number in asn_list:
    data = get_asn_data(url, asn_number)
    name = data['data']['asns'][0]['description']
    abuse_contacts = data['data']['asns'][0]['abuse_contacts'][0]
    rir = data['data']['asns'][0]['rir_name']
    print(f'Name: {name}')
    print(f'Abuse contact: {abuse_contacts}')
    print(f'RIR name: {rir}')
    print('-------------------------')

Name: Digital Ocean, Inc.
Abuse contact: abuse@digitalocean.com
RIR name: APNIC
-------------------------
Name: Alviva Holding Limited
Abuse contact: abuse@digi-cloud.net
RIR name: RIPE
-------------------------
Name: Hetzner Online GmbH
Abuse contact: abuse@hetzner.com
RIR name: RIPE
-------------------------
Name: Base IP B.V.
Abuse contact: abuse@baseip.com
RIR name: RIPE
-------------------------


Stran: https://rapidapi.com/ -> tržnica APijev